# Merge and early preprocessing

### TODO 

Add component features in addition to or instead of total features (SUSPENDED)

Discuss Raw score vs Scaled/T scores with Andrijana

Discuss SDMT/BLJO situation with andrijana

Discuss the decision of just treating all ordinal variables as numeric ones.  Justification: we already do this when we sum them up in total scores which we then consider numeric

#### This cell is for defining various OPTIONS used for this notebook (working directory, how many rows and columns pandas displays for a dataframe, etc). 

#### Preferably this cell is also where we do important imports (for example pandas and numpy)

In [41]:
import os 
#Input the directory where your database is located 
#os.chdir('C:/Users/Trond/Documents/Master 2020/Parkinson_PPMI')
os.chdir('C:/Users/Briggstone/Documents/Master 2020/Parkinson_PPMI')
#os.chdir('C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Parkinson_PPMI')

#Where you want the csv file of the merged data to be placed
#output_filepath = 'C:/Users/Trond/Documents/Master 2020/Processed data'
output_filepath = 'C:/Users/Briggstone/Documents/Master 2020/Processed data'
#output_filepath = 'C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data'

import pandas as pd 
import numpy as np

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 150
pd.set_option('display.max_columns', 150)




#### In this cell we select the subset of patients with parkinson's that have valid enrollment dates

In [42]:
#Reading the data table Screening demographics into a pandas dataframe
data = pd.read_csv('_Subject_Characteristics/Screening___Demographics.csv') 
print("Number of total subjects in original data table: ", data.shape[0])
data2 = pd.read_csv('Study_Enrollment/Randomization_table.csv')
data = data.merge(data2, how = "inner", on = "PATNO")

#Discarding indviduals that do not have confirmed parkinson's, Everyone except APPRDX == 1. Also making sure that the subject has a valid enrollment date in the project
data = data.loc[((data.APPRDX == 1) | (data.APPRDX == 5))  & (data.ENROLLDT.notna())]
print("Number of subjects with parkinson's in data table:", data.shape[0])


#Selecting only PATNO, APPRDX and ENROLLDT columns and reseting index (0-1-2 instead of 2-6-9). Index was weird due to dropping columns in the previous lines. 
pat_subset = pd.DataFrame(data.loc[:, ["PATNO", "ENROLLDT", "APPRDX"]].reset_index(drop = True))

Number of total subjects in original data table:  2200
Number of subjects with parkinson's in data table: 683


#### In this cell we import our data tables into pandas DataFrames and select the features we want from each data table. 

In [43]:
#Importing data tables from non-motor and selecting relevant columns
MOCA = pd.read_csv("Non-motor_Assessments/Montreal_Cognitive_Assessment__MoCA_.csv").loc[:,["PATNO", "EVENT_ID", "MCAALTTM", "MCACUBE", \
    "MCACLCKC", "MCACLCKN", "MCACLCKH", "MCALION", "MCARHINO", "MCACAMEL", "MCAFDS", "MCABDS", "MCAVIGIL", "MCASER7", "MCASNTNC", "MCAVF", \
    "MCAABSTR", "MCAREC1", "MCAREC2", "MCAREC3", "MCAREC4", "MCAREC5", "MCADATE", "MCAMONTH", "MCAYR", "MCADAY", "MCAPLACE", "MCACITY"]]

HVLT = pd.read_csv("Non-motor_Assessments/Hopkins_Verbal_Learning_Test.csv").loc[:,["PATNO", "EVENT_ID", "HVLTRT1", "HVLTRT2", \
        "HVLTRT3", "HVLTRDLY", "HVLTREC", "HVLTFPRL", "HVLTFPUN"]]

BJLO = pd.read_csv("Non-motor_Assessments/Benton_Judgment_of_Line_Orientation.csv").loc[:,["PATNO", "EVENT_ID", "JLO_TOTRAW"]]

LNSQ = pd.read_csv("Non-motor_Assessments/Letter_-_Number_Sequencing__PD_.csv").loc[:,["PATNO", "EVENT_ID", "LNS_TOTRAW"]]

SEFL = pd.read_csv("Non-motor_Assessments/Semantic_Fluency.csv").loc[:,["PATNO", "EVENT_ID", "VLTANIM", "VLTVEG", "VLTFRUIT"]]

REMQ = pd.read_csv("Non-motor_Assessments/REM_Sleep_Disorder_Questionnaire.csv").loc[:,["PATNO", "EVENT_ID", "DRMVIVID", "DRMAGRAC", \
    "DRMNOCTB", "SLPLMBMV", "SLPINJUR", "DRMVERBL", "DRMFIGHT", "DRMUMV", "DRMOBJFL", "MVAWAKEN", "DRMREMEM", "SLPDSTRB", "STROKE", \
    "HETRA", "PARKISM", "RLS", "NARCLPSY", "DEPRS", "EPILEPSY", "BRNINFM", "CNSOTH"]]

GDSS = pd.read_csv("Non-motor_Assessments/Geriatric_Depression_Scale__Short_.csv").loc[:,["PATNO", "EVENT_ID", "GDSSATIS", "GDSDROPD", \
    "GDSEMPTY", "GDSBORED", "GDSGSPIR", "GDSAFRAD", "GDSHAPPY", "GDSHLPLS", "GDSHOME", "GDSMEMRY", "GDSALIVE", "GDSWRTLS", "GDSENRGY", \
    "GDSHOPLS", "GDSBETER"]]

SIDT = pd.read_csv("Non-motor_Assessments/University_of_Pennsylvania_Smell_ID_Test.csv").loc[:,["PATNO", "UPSITBK1", "UPSITBK2", \
    "UPSITBK3", "UPSITBK4"]]

EPSS = pd.read_csv("Non-motor_Assessments/Epworth_Sleepiness_Scale.csv").loc[:,["PATNO", "EVENT_ID", "ESS1", "ESS2", \
    "ESS3", "ESS4", "ESS5", "ESS6", "ESS7", "ESS8"]]

SCOP = pd.read_csv("Non-motor_Assessments/SCOPA-AUT.csv").loc[:,["PATNO", "EVENT_ID", "SCAU1", "SCAU2", \
    "SCAU3", "SCAU4", "SCAU5", "SCAU6", "SCAU7", "SCAU8", "SCAU9", "SCAU10", "SCAU11", "SCAU12", "SCAU13", \
    "SCAU14", "SCAU15", "SCAU16", "SCAU17", "SCAU18", "SCAU19", "SCAU20", "SCAU21", "SCAU22", "SCAU23", "SCAU24", "SCAU25"]]

SDIM = pd.read_csv("Non-motor_Assessments/Symbol_Digit_Modalities.csv").loc[:,["PATNO", "EVENT_ID", "SDMTOTAL"]]


#Importing data tables from motor
MSU3 = pd.read_csv("Motor___MDS-UPDRS/MDS_UPDRS_Part_III.csv").loc[:,["PATNO", "EVENT_ID", 'NP3BRADY', 'NP3FACXP', 'NP3FRZGT', \
    'NP3FTAPL', 'NP3FTAPR', 'NP3GAIT', 'NP3HMOVL', 'NP3HMOVR', 'NP3KTRML', 'NP3KTRMR', 'NP3LGAGL', 'NP3LGAGR', 'NP3POSTR', 'NP3PRSPL', 'NP3PRSPR', \
    'NP3PSTBL', 'NP3PTRML', 'NP3PTRMR', 'NP3RIGLL', 'NP3RIGLU', 'PN3RIGRL', 'NP3RIGN', 'NP3RIGRU', 'NP3RISNG', 'NP3RTALJ', 'NP3RTALL', 'NP3RTALU', \
    'NP3RTARL', 'NP3RTARU', 'NP3RTCON', 'NP3SPCH', 'NP3TTAPL', 'NP3TTAPR', 'PAG_NAME', 'PD_MED_USE', 'NHY']]
MSU1 = pd.read_csv("Motor___MDS-UPDRS/MDS_UPDRS_Part_I.csv").loc[:,["PATNO", "EVENT_ID", "NP1HALL", "NP1COG"]]
MSU2 = pd.read_csv ("Motor___MDS-UPDRS/MDS_UPDRS_Part_II__Patient_Questionnaire.csv").loc[:,["PATNO", "EVENT_ID", "NP2TRMR", "NP2WALK", "NP2FREZ"]]

#Importing data tables from subject characteristics
FMHS = pd.read_csv("_Subject_Characteristics/Family_History__PD_.csv").loc[:,["PATNO", "BIOMOMPD", "BIODADPD", \
    "FULSIBPD", "HAFSIBPD", "MAGPARPD", "PAGPARPD", "MATAUPD", "PATAUPD", "KIDSPD"]]
SOEC = pd.read_csv("_Subject_Characteristics/Socio-Economics.csv").loc[:,["PATNO", "EDUCYRS"]]
SCDE = pd.read_csv("_Subject_Characteristics/Screening___Demographics.csv").loc[:,["PATNO", "BIRTHDT", "GENDER"]]
MDHI = pd.read_csv("Medical_History/PD_Features.csv").loc[:,["PATNO", "PDDXDT"]]

#### In this cell we do any additional preprocessing on the data before we perform our data merge

In [44]:
# SC to BL for MOCA as it has no Baseline data
MOCA.loc[MOCA["EVENT_ID"] == "SC", "EVENT_ID"] = "BL"

#Drop MSU3 data entries that are post drug-administering

MSU3 = MSU3.drop(MSU3[ MSU3['PAG_NAME'] == "NUPDRS3A" ].index)
MSU3 = MSU3.drop("PAG_NAME", axis = 1)

#For Gender we need to combine 0 (Female of child bearing potential) and 1 (Female of non-child bearing potential) into one category 0 (female), we convert 2 (Male) to 1 afterwards
#Result of this processing: 0 = Female, 1 = Male

SCDE.loc[SCDE["GENDER"] == 1, "GENDER"] = 0 
SCDE.loc[SCDE["GENDER"] == 2, "GENDER"] = 1

#SCOP is not a simple sum and needs additional preprocessing to get total score
# In SCAU1-21, 9 is converted to 3. In SCAU22-25, 9 is converted to 0
for i in range(1,26):
    s = "SCAU" + str(i)
    points = 0
    if i < 22:
        points = 3
    SCOP.loc[SCOP[s] == 9, s] = points

#### In this cell we perform our data merge

In [45]:
# First step of join: Merge non-longitudinal data
data = pat_subset.merge(FMHS, how = "inner", on = "PATNO")
data = data.merge(SOEC, how = "inner", on = "PATNO")
data = data.merge(SCDE, how = "inner", on = "PATNO")
data = data.merge(SIDT, how = "inner", on = "PATNO")
data = data.merge(MDHI, how = "inner", on = "PATNO")

#Second step of join: Merge in longitudinal data

#This data table has duplicate entries for the same data, test is done before drug is administered and after
data = data.merge(MSU3, how = "inner", on ="PATNO")

data = data.merge(SDIM, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(MSU1, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(MOCA, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(HVLT, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(BJLO, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(LNSQ, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(SEFL, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(GDSS, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(EPSS, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(SCOP, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(REMQ, how = "inner", on = ["PATNO", "EVENT_ID"])
data = data.merge(MSU2, how = "inner", on = ["PATNO", "EVENT_ID"])



#### In this cell we do some additional processing post merge and output the merged data table to a csv file so that it can be further processed in missingvalues.ipynb

In [46]:
#Removing birthdate, date of Parkinson's disease diagnosis and project enrollment date and adding age and PD duration as a function of these columns. 

#Function for month/year accuracy of PD_DURATION_BL
def pd_duration(row):
    
    enroll = float(row["ENROLLDT"][0:2]) / 12 + float(row["ENROLLDT"][-4:])
    pdstart = float(row["PDDXDT"][0:2]) / 12 + float(row["PDDXDT"][-4:])
    
    return enroll - pdstart
       
data ["AGE"] = data.apply(lambda row_wise: int(row_wise["ENROLLDT"][-4:]) - int(row_wise["BIRTHDT"]) , axis = 1)
data["PD_DURATION"] = data.apply(pd_duration, axis = 1)


#Updating AGE and PD_Duration for subjects by 1 year for each year in the study
year_dict = {
    "BL" : 0,
    "V04" : 1,
    "V06" : 2,
    "V08" : 3,
    "V10" : 4,
    "V12" : 5,
    "V13" : 6,
    "V14" : 7,
    "V15" : 8    
}

for k,v in year_dict.items():
    data.loc[data.EVENT_ID == k, "AGE"] = data.loc[data.EVENT_ID == k, "AGE"] + v
    data.loc[data.EVENT_ID == k, "PD_DURATION"] = data.loc[data.EVENT_ID == k, "PD_DURATION"] + v       
        
                            
                            

data = data.drop(["ENROLLDT", "BIRTHDT", "PDDXDT"], axis = 1)


#Filtering out patients from genetic cohort that do not meet the inclusion criteria and then removing APPRDX as it no longer needed
valid_subjects = data.loc[(data.APPRDX == 1) | ((data.APPRDX == 5) & (data.AGE > 30) & (data.PD_DURATION <= 2) & (data.NHY < 3)  & (data.EVENT_ID == "BL") & \
                (data.PD_MED_USE == 0))].PATNO.unique()

for x in data.PATNO.unique():
    if x not in valid_subjects:
        data = data[data.PATNO != x]

data = data.drop("APPRDX", axis = 1)


#Dropping all PATNO who have hallucinations at baseline, NP1HALL != 0

subjects_hall_BL = data.loc[(data.EVENT_ID == "BL") & (data.NP1HALL != 0), "PATNO"].unique()

for x in subjects_hall_BL:
    data = data[data.PATNO != x]
    
'''
We are only interested in event_ids [BL,V04,V06,V08,V10,V12,V13,V14,V15] as these are the datapoints we can order in time and that has complete data
We therefore drop datapoints that do not have these event_ids
'''
valid_event_ids = ["BL", "V04", "V06", "V08", "V10", "V12", "V13", "V14", "V15"]
event_ids_to_remove = np.setdiff1d(data.EVENT_ID.unique(), valid_event_ids)

for x in event_ids_to_remove:
    data = data[data.EVENT_ID != x]

    
#Adding a useful column which tells us the last data point a subject has in the study

data["LAST_EVENT_ID"] = np.nan
for x in data.PATNO.unique():
    subject_event_ids = data.loc[data.PATNO == x, "EVENT_ID"].unique()
    data.loc[data.PATNO == x, ["LAST_EVENT_ID"]] = (np.sort(subject_event_ids))[-1]


#merged data table to file
data.to_csv(output_filepath + '/joined_data.csv', index = False)



### In this cell we can do any final data inspection (Number of unique patients, shape, etc)

In [47]:
import numpy as np
#Inspecting the merged data

#np.sort(data.columns.values)
print( "Data shape", data.shape)
print( "Number of unique PATNO:", data.PATNO.unique().size)

data.head(10)

Data shape (2258, 168)
Number of unique PATNO: 418


,PATNO,BIOMOMPD,BIODADPD,FULSIBPD,HAFSIBPD,MAGPARPD,PAGPARPD,MATAUPD,PATAUPD,KIDSPD,EDUCYRS,GENDER,UPSITBK1,UPSITBK2,UPSITBK3,UPSITBK4,EVENT_ID,NP3BRADY,NP3FACXP,NP3FRZGT,NP3FTAPL,NP3FTAPR,NP3GAIT,NP3HMOVL,NP3HMOVR,NP3KTRML,NP3KTRMR,NP3LGAGL,NP3LGAGR,NP3POSTR,NP3PRSPL,NP3PRSPR,NP3PSTBL,NP3PTRML,NP3PTRMR,NP3RIGLL,NP3RIGLU,PN3RIGRL,NP3RIGN,NP3RIGRU,NP3RISNG,NP3RTALJ,NP3RTALL,NP3RTALU,NP3RTARL,NP3RTARU,NP3RTCON,NP3SPCH,NP3TTAPL,NP3TTAPR,PD_MED_USE,NHY,SDMTOTAL,NP1HALL,NP1COG,MCAALTTM,MCACUBE,MCACLCKC,MCACLCKN,MCACLCKH,MCALION,MCARHINO,MCACAMEL,MCAFDS,MCABDS,MCAVIGIL,MCASER7,MCASNTNC,MCAVF,MCAABSTR,MCAREC1,MCAREC2,MCAREC3,MCAREC4,MCAREC5,...,GDSSATIS,GDSDROPD,GDSEMPTY,GDSBORED,GDSGSPIR,GDSAFRAD,GDSHAPPY,GDSHLPLS,GDSHOME,GDSMEMRY,GDSALIVE,GDSWRTLS,GDSENRGY,GDSHOPLS,GDSBETER,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,SCAU1,SCAU2,SCAU3,SCAU4,SCAU5,SCAU6,SCAU7,SCAU8,SCAU9,SCAU10,SCAU11,SCAU12,SCAU13,SCAU14,SCAU15,SCAU16,SCAU17,SCAU18,SCAU19,SCAU20,SCAU21,SCAU22,SCAU23,SCAU24,SCAU25,DRMVIVID,DRMAGRAC,DRMNOCTB,SLPLMBMV,SLPINJUR,DRMVERBL,DRMFIGHT,DRMUMV,DRMOBJFL,MVAWAKEN,DRMREMEM,SLPDSTRB,STROKE,HETRA,PARKISM,RLS,NARCLPSY,DEPRS,EPILEPSY,BRNINFM,CNSOTH,NP2TRMR,NP2WALK,NP2FREZ,AGE,PD_DURATION,LAST_EVENT_ID
0,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,0.0,6.0,8.0,7.0,9.0,BL,0.0,1.0,0.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0,1.0,47.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,3.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,39,0.750000,V15
1,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,0.0,6.0,8.0,7.0,9.0,V04,3.0,2.0,0.0,3.0,2.0,1.0,3.0,2.0,1.0,0.0,3.0,2.0,0.0,3.0,2.0,0.0,1.0,0.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,3,2.0,45.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,3.0,3.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,40,1.750000,V15
2,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,0.0,6.0,8.0,7.0,9.0,V06,2.0,1.0,0.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,1.0,2,2.0,47.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,3.0,3.0,0.0,0.0,2.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,41,2.750000,V15
3,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,0.0,6.0,8.0,7.0,9.0,V12,2.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,4,2.0,34.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,3.0,3.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,44,5.750000,V15
4,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,0.0,6.0,8.0,7.0,9.0,V08,3.0,3.0,0.0,3.0,2.0,2.0,3.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,6,2.0,47.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,